In [1]:
!git clone https://github.com/Q2AD/Q2AD.git

fatal: destination path 'Q2AD' already exists and is not an empty directory.


In [2]:
import json

with open('Q2AD/Q2AD_v1_QuestionsID.json', 'r') as f:
    questions = json.load(f)

with open('Q2AD/Q2AD_v1_train.json', 'r') as f:
    train = json.load(f)

with open('Q2AD/Q2AD_v1_test.json', 'r') as f:
    test = json.load(f)


In [3]:
train = [
    {
        'question': questions[question_id],
        'answer': item['full_text'],
    }
    for question_id, item in train.items()
]

test = [
    {
        'question': questions[question_id],
        'answer': item['full_text'],
    }
    for question_id, item in test.items()
]

In [14]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained('sshleifer/distilbart-xsum-12-3')
model = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-xsum-12-3')

loading file vocab.json from cache at /home/ubuntu/.cache/huggingface/hub/models--sshleifer--distilbart-xsum-12-3/snapshots/1d2bfbc16dcdd28720f9f1d37be764e5cc5c78c8/vocab.json
loading file merges.txt from cache at /home/ubuntu/.cache/huggingface/hub/models--sshleifer--distilbart-xsum-12-3/snapshots/1d2bfbc16dcdd28720f9f1d37be764e5cc5c78c8/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/ubuntu/.cache/huggingface/hub/models--sshleifer--distilbart-xsum-12-3/snapshots/1d2bfbc16dcdd28720f9f1d37be764e5cc5c78c8/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--sshleifer--distilbart-xsum-12-3/snapshots/1d2bfbc16dcdd28720f9f1d37be764e5cc5c78c8/tokenizer_config.json
loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--sshleifer--distilbart-xsum-12-3/snapshots/1d2bfbc16dcdd28720f9f1d37be764e5cc5c78c8/config.json
Model 

In [15]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=2,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  # demo
    per_device_eval_batch_size=4,
    learning_rate=1e-04,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [16]:
from torch.utils.data.dataloader import DataLoader


def preprocess(data):
    new_data = []
    for item in data:
        tokenized = tokenizer(item['answer'], return_tensors='pt', padding='max_length', truncation=True, max_length=512)
        tokenized_question = tokenizer(item['question'], return_tensors='pt', padding='max_length', truncation=True, max_length=512)
        labels = tokenized_question['input_ids'][0]
        labels[labels == tokenizer.pad_token_id] = -100
        new_data.append({
            'input_ids': tokenized['input_ids'][0],
            'attention_mask': tokenized['attention_mask'][0],
            'labels': labels
        })
    return new_data

train_preprocess = preprocess(train)
test_preprocess = preprocess(test)

train_loader = list(DataLoader(train_preprocess, batch_size=training_args.per_device_train_batch_size, shuffle=True))
test_loader = list(DataLoader(test_preprocess, batch_size=training_args.per_device_eval_batch_size, shuffle=False))


In [17]:
def compute_metrics(eval_preds):
    pass

In [18]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_preprocess,
    eval_dataset=test_preprocess,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [19]:
trainer.train()

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 240
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 120
  Number of trainable parameters = 255120384


Step,Training Loss
50,5.442300
100,4.382500




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=120, training_loss=4.760571734110514, metrics={'train_runtime': 121.5242, 'train_samples_per_second': 3.95, 'train_steps_per_second': 0.987, 'total_flos': 297195798528000.0, 'train_loss': 4.760571734110514, 'epoch': 2.0})

In [20]:
for i in range(10):
    encoding = tokenizer(test[i]['answer'], return_tensors='pt', padding='max_length', truncation=True, max_length=512)
    input_ids = encoding['input_ids'].to('cuda')
    attention_mask = encoding['attention_mask'].to('cuda')
    result = model.generate(input_ids, attention_mask=attention_mask, max_length=512, num_beams=4, early_stopping=True)
    prediction = tokenizer.decode(result[0], skip_special_tokens=True)
    print('Question: ', test[i]['question'])
    print('Prediction: ', prediction)
    print('Answer: ', test[i]['answer'])
    print('--------------------------------')

Question:  What sport should I try out?
Prediction:  What is one of the most popular sports in the world?
Answer:  Table tennis is tremendously fun, and a much better source of exercise than people tend to give it credit for. It’s simultaneously a very intense mental sport and one that engages a lot of different muscle groups. As an added plus, people who don’t play it will think you’re amazing if you can play the game at anywhere above a novice level!
--------------------------------
Question:  What is the point of a tattoo?
Prediction:  What would you do if you got a tattoo?
Answer:  There are a variety of reasons for getting a tattoo. You can get one to remember someone/something, you can get one because it looks pretty or cool, or a variety of other reasons. Reasons will certainly differ from person to person. I got my tattoo to remember my grandmother, the depression I experienced after her death, and to celebrate how my partner helped me get through that dark time.
--------------

: 